In [1]:
# Cell 1: Import libraries and set up directories
import os
import cv2
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# # Cell 2: Extract frames from videos
# def extract_frames(video_path, output_folder, label):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)
    
#     cap = cv2.VideoCapture(video_path)
#     frame_count = 0
    
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         frame_name = os.path.join(output_folder, f"{label}_{frame_count}.jpg")
#         cv2.imwrite(frame_name, frame)
#         frame_count += 1
    
#     cap.release()
#     cv2.destroyAllWindows()

# video_folders = {
#     "violence": "Dataset/Violence",
#     "non_violence": "Dataset/NonViolence"
# }

# output_folder = "extract"

# for label, folder in video_folders.items():
#     for video_file in os.listdir(folder):
#         video_path = os.path.join(folder, video_file)
#         extract_frames(video_path, os.path.join(output_folder, label), label)


In [3]:
# Cell 3: Extract frames from videos
def prepare_yolo_data(frame_folder, labels):
    image_files = []
    for label in labels:
        label_folder = os.path.join(frame_folder, label)
        for file in os.listdir(label_folder):
            if file.endswith(".jpg"):
                image_files.append((os.path.join(label_folder, file), labels.index(label)))

    df = pd.DataFrame(image_files, columns=["image_path", "label"])
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"])

    def save_labels(df, split):
        os.makedirs(f"data/{split}/images", exist_ok=True)
        os.makedirs(f"data/{split}/labels", exist_ok=True)

        for _, row in df.iterrows():
            image_path, label = row["image_path"], row["label"]
            base_name = os.path.basename(image_path)
            new_image_path = f"data/{split}/images/{base_name}"
            label_path = new_image_path.replace("images", "labels").replace(".jpg", ".txt")

            if os.path.exists(new_image_path):
                print(f"File {new_image_path} already exists. Skipping.")
                continue
            try:
                shutil.move(image_path, new_image_path)
            except FileExistsError:
                print(f"File {new_image_path} already exists. Skipping.")
                continue
            with open(label_path, "w") as f:
                f.write(f"{label} 0.5 0.5 1 1\n")

    save_labels(train_df, "train")
    save_labels(val_df, "val")

prepare_yolo_data("extract", ["non_violence", "violence"])

File data/train/images/violence_179.jpg already exists. Skipping.
File data/train/images/violence_26.jpg already exists. Skipping.
File data/train/images/violence_157.jpg already exists. Skipping.
File data/train/images/violence_69.jpg already exists. Skipping.
File data/train/images/violence_58.jpg already exists. Skipping.
File data/train/images/violence_34.jpg already exists. Skipping.
File data/train/images/violence_165.jpg already exists. Skipping.
File data/train/images/violence_64.jpg already exists. Skipping.
File data/train/images/violence_82.jpg already exists. Skipping.
File data/train/images/violence_200.jpg already exists. Skipping.
File data/train/images/violence_35.jpg already exists. Skipping.
File data/train/images/violence_138.jpg already exists. Skipping.
File data/train/images/violence_184.jpg already exists. Skipping.
File data/train/images/violence_90.jpg already exists. Skipping.
File data/train/images/violence_118.jpg already exists. Skipping.
File data/train/im

In [4]:
# Cell 4: Save data configuration
data_config = {
    'path': 'data',  
    'train': 'train/images', 
    'val': 'val/images',  
    'names': ['non_violence', 'violence']  
}

import yaml

with open('data_config.yaml', 'w') as f:
    yaml.dump(data_config, f)

In [5]:
# Cell 5: Train YOLOv8 model
from ultralytics import YOLO
model = YOLO('Models/yolov8n.pt') 
model.train(data='data_config.yaml', epochs=25, imgsz=640)
model.save('Models/violence_det.pt')

New https://pypi.org/project/ultralytics/8.2.74 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.17 🚀 Python-3.12.1 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i7-1255U)
engine\trainer: task=detect, mode=train, model=Models/yolov8n.pt, data=data_config.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

train: Scanning C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\train\labels... 13378 images, 0 backgrounds, 1 corrupt: 100%|██████████| 13379/13379 [00:09<00:00, 1454.36it/s]

train: WARNING ⚠️ C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\train\images\violence_7.jpg: ignoring corrupt image/label: image file is truncated (38 bytes not processed)


train: New cache created: C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\train\labels.cache


val: Scanning C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\val\labels... 3396 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3396/3396 [00:02<00:00, 1515.55it/s]


val: New cache created: C:\Users\sonit\OneDrive\Desktop\Work\Coding\Violence-Detection\data\val\labels.cache
Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train10
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200         0G      1.194      2.996      1.736         53        640:  12%|█▏        | 99/837 [12:53<1:36:05,  7.81s/it]


KeyboardInterrupt: 